# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Lab 5</font>
## <font color='blue'>Chatbot Personalizado com IA Generativa e LangChain</font>

![DSA](imagens/Lab5.png)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00


In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.8 MB/s eta 0:00:00


In [4]:
# Imports
import torch
import langchain
import transformers
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Markdown

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

langchain   : 0.0.228
transformers: 4.30.2
torch       : 2.0.1+cu118



In [6]:
# Checando a GPU
!nvidia-smi

Sat Jul  8 22:07:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Definindo o LLM Para IA Generativa

Este Lab usa o modelo de linguagem **bertin-project/bertin-gpt-j-6B-alpaca** do HuggingFace para gerar respostas aprimoradas com base em uma sequência de texto inicial.

O principal objetivo é melhorar as respostas geradas pelo modelo **bertin-project/bertin-gpt-j-6B-alpaca** usando técnicas como Prompt Templates e Sequential Chains.

O **bertin-project/bertin-gpt-j-6B-alpaca** é um modelo de linguagem de inteligência artificial baseado no modelo GPT-J 6B. É uma versão em espanhol deste modelo que foi treinada para gerar texto em espanhol. O modelo foi treinado usando o conjunto de dados Alpaca, que é uma versão limpa do conjunto de dados Alpaca criado em Stanford.

https://huggingface.co/bertin-project/bertin-gpt-j-6B-alpaca

In [7]:
# Repositório do modelo
modelo = "bertin-project/bertin-gpt-j-6B-alpaca"

In [8]:
# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(modelo)

In [9]:
# Cria o pipeline
pipeline = pipeline("text-generation",

                    # Modelo que será usado para geração de texto
                    model = modelo,

                    # Tokenizer
                    tokenizer = tokenizer,

                    # Tipo dos dados para computação
                    torch_dtype = torch.bfloat16,

                    # Se devemos confiar no código remoto ao carregar o modelo.
                    # Isso só deve ser definido como True se você confiar na origem do modelo.
                    trust_remote_code = True,

                    # O dispositivo a ser usado para os cálculos do modelo.
                    device_map = "auto",

                    # O comprimento máximo do texto a ser gerado.
                    max_length = 512,

                    # Se deve usar a amostragem ao gerar texto.
                    do_sample = True,

                    # O número de candidatos a manter durante a amostragem.
                    top_k = 2,

                    # O número de sequências de texto a serem geradas.
                    num_return_sequences = 1,

                    # O ID do token de fim de sequência.
                    eos_token_id = tokenizer.eos_token_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

A temperatura é um hiperparâmetro em muitos modelos de linguagem que controla a aleatoriedade da saída do modelo. Um valor de temperatura mais baixo (como 0.2) faz com que o modelo produza saídas mais determinísticas, enquanto um valor de temperatura mais alto faz com que o modelo produza saídas mais aleatórias.

In [10]:
# HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {"temperature": 0.2})

### Testando o Modelo Sem o Uso de Templates (Direto com Prompts)

In [11]:
def imprime_resposta(string):
    display(Markdown(string))

In [12]:
imprime_resposta(llm("Escribe un poema sobre el sol"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.

### Respuesta:
El sol es una maravilla cósmica, 
Brillando y brillando, sin cesar. 
Su calor y luz nos dan vida, 
Iluminando cada día, con gran esplendor. 
Es una fuente de energía, 
Que nos da la oportunidad de volar, 
Y hacer nuestra propia suerte, 
En este mundo, tan grandioso y ancho.

In [13]:
imprime_resposta(llm("Puedes decirme qué es una computadora?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




### Respuesta:
Una computadora es un dispositivo que almacena, recupera y procesa datos. Es un sistema informático que consta de componentes físicos y software, y es capaz de realizar operaciones a nivel digital. Las computadoras pueden ser utilizadas para una variedad de tareas, desde simples cálculos hasta tareas sofisticadas de aprendizaje automático.

In [14]:
imprime_resposta(llm("Me explicarias que es la inteligencia artificial?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




### Respuesta:
La inteligencia artificial es un área de la informática que se enfoca en crear máquinas que puedan pensar y actuar como los humanos, especialmente en tareas complejas. Involucra el uso de algoritmos avanzados para analizar grandes cantidades de datos, reconocer patrones y tomar decisiones similares a las que tomaría una persona. La IA se puede utilizar en una variedad de campos, desde la atención médica hasta los negocios y la defensa.

### Testando o Modelo com Prompt Templates do LangChain

In [ ]:
#langchain.debug = True

> Exemplo 1

In [15]:
# Template
template = """
  Eres un chatbot experto en psicologia y neurociencia.
  Responde a la siguiente pregunta con respuestas brillantes.
  Sólo tienes que escribir la respuesta, no más.
  Pregunta: {pregunta}
  Respuesta:
"""

In [16]:
# Prompt
prompt = PromptTemplate(template = template, input_variables = ["pregunta"])
print(prompt)

input_variables=['pregunta'] output_parser=None partial_variables={} template='\n  Eres un chatbot experto en psicologia y neurociencia.\n  Responde a la siguiente pregunta con respuestas brillantes.\n  Sólo tienes que escribir la respuesta, no más.\n  Pregunta: {pregunta}\n  Respuesta:\n' template_format='f-string' validate_template=True


In [17]:
# Chain
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [18]:
# Questão
question = "Explicame que es la atencion?"

In [19]:
# Executa a chain
exec_exemplo1 = llm_chain.run(question)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
# Print
imprime_resposta(exec_exemplo1)

La atención es la concentración o enfoque de la mente en una tarea o actividad. Es un proceso que nos permite prestar atención a algo y mantenernos enfocados en ello, a pesar de cualquier distracción. La atención es esencial para el aprendizaje, ya que nos permite recopilar, organizar y recordar información. También nos ayuda a mantenernos conscientes y a estar presentes en el momento.

> Exemplo 2

In [21]:
# Template
template = """
  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.
  Solo tienes responder a la pregunta no más.
  Pregunta: {pregunta}
  Respuesta:
"""

In [22]:
# Prompt
prompt = PromptTemplate(template = template, input_variables = ["pregunta"])
print(prompt)

input_variables=['pregunta'] output_parser=None partial_variables={} template='\n  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.\n  Solo tienes responder a la pregunta no más.\n  Pregunta: {pregunta}\n  Respuesta:\n' template_format='f-string' validate_template=True


In [23]:
# Chain
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [24]:
# Questão
question = "Que es un diccionario?"

In [25]:
# Executa a chain
exec_exemplo2 = llm_chain.run(question)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
imprime_resposta(exec_exemplo2)

  Un diccionario es una estructura de datos que almacena pares de clave-valor. Es una forma eficiente de almacenar información estructurada, como palabras y sus definiciones. En Python, un diccionario se crea con la sintaxis 'dict', seguido del nombre que deseas darle al diccionario. Por ejemplo, si quisieras crear un diccionario que almacenara las palabras "feliz", "triste" y "enojado", podrías hacerlo con la sintaxis 'dict["feliz","triste","enojado"]:'.

In [27]:
# Questão
question = "Cuales son los tipos de datos basicos en python?"

In [28]:
imprime_resposta(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


En Python, los tipos básicos de datos incluyen cadenas de texto, números, boletos, listas y tuplas. Además, Python tiene varios tipos de datos definidos por el usuario, como matrices, objetos e incluso bibliotecas.

## Sequential Chains

In [29]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "pregunta: Cual es el mejor nombre para describir una empresa que fabrica {producto}, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta:"
)

In [30]:
# Chain 1
chain_one = LLMChain(llm = llm, prompt = first_prompt)

In [31]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Escribe una descripcion creativa de la siguiente compañia: {company_name}"
)

In [32]:
# chain 2
chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [33]:
# SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains = [chain_one, chain_two], verbose = True)

In [34]:
product = "Burritos veganos"

In [35]:
exec_chain = overall_simple_chain.run(product)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new  chain...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Burritos Veganos Gritones.


### Respuesta:
Burritos Veganos Gritones es una deliciosa y creativa empresa que se especializa en deliciosos y saludables burritos veganos. Utilizando solo los ingredientes y sabores más frescos, los burritos de Burritos Veganos Gritones están hechos para satisfacer todos los paladares. Desde el clásico Burrito de Tres Frijoles hasta el exótico Burrito de Cuatro Frijoles, cada uno está hecho con amor para brindarte una experiencia única. Ya sea que estés buscando una comida saludable y llenadora o una opción vegana, los burritos de Burritos Veganos Gritones son la elección perfecta.

> Finished chain.


In [36]:
imprime_resposta(exec_chain)



### Respuesta:
Burritos Veganos Gritones es una deliciosa y creativa empresa que se especializa en deliciosos y saludables burritos veganos. Utilizando solo los ingredientes y sabores más frescos, los burritos de Burritos Veganos Gritones están hechos para satisfacer todos los paladares. Desde el clásico Burrito de Tres Frijoles hasta el exótico Burrito de Cuatro Frijoles, cada uno está hecho con amor para brindarte una experiencia única. Ya sea que estés buscando una comida saludable y llenadora o una opción vegana, los burritos de Burritos Veganos Gritones son la elección perfecta.

# Fim